In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import tensorflow as tf

iris = load_iris()
# shuffle = True 随机打乱后再进行分割数据
X_train, X_test, ori_y_train, ori_y_test = train_test_split(iris.data, iris.target, test_size=0.2, shuffle=True) 
# 独热编码 0用[1,0,0]表示, 1用[0,1,0]表示，这样的只含有0，1的数组
y_train = tf.one_hot(ori_y_train, 3)
y_test = tf.one_hot(ori_y_test, 3)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
# 数据输入 → 全连层*3 → Dropout层（防止过拟合） → 3分类输出
model = Sequential()
model.add(Dense(1024, input_dim=4, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              5120      
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 662,019
Trainable params: 662,019
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

Epoch 1/50
4/4 [==============================] - 1s 45ms/step - loss: 1.0991 - accuracy: 0.4167 - val_loss: 0.8402 - val_accuracy: 0.6000
Epoch 2/50
4/4 [==============================] - 0s 12ms/step - loss: 0.7088 - accuracy: 0.7417 - val_loss: 0.5187 - val_accuracy: 0.7333
Epoch 3/50
4/4 [==============================] - 0s 12ms/step - loss: 0.5029 - accuracy: 0.7000 - val_loss: 0.3780 - val_accuracy: 0.8667
Epoch 4/50
4/4 [==============================] - 0s 12ms/step - loss: 0.3817 - accuracy: 0.8417 - val_loss: 0.2938 - val_accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 0s 13ms/step - loss: 0.2950 - accuracy: 0.9167 - val_loss: 0.2894 - val_accuracy: 0.8667
Epoch 6/50
4/4 [==============================] - 0s 12ms/step - loss: 0.2419 - accuracy: 0.9000 - val_loss: 0.2311 - val_accuracy: 0.8667
Epoch 7/50
4/4 [==============================] - 0s 13ms/step - loss: 0.2043 - accuracy: 0.9250 - val_loss: 0.4428 - val_accuracy: 0.7667
Epoch 8/50
4/4 [===========

In [4]:
import pickle

# 模型预测
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)
# 训练记录保存
with open('./data/irises_history.txt', 'wb', ) as file_txt:
    pickle.dump(history.history, file_txt)
# 模型保存.h5
model.save('./model/irises_classifier_pro.h5')

1/1 [==============================] - 0s 18ms/step - loss: 0.0493 - accuracy: 1.0000
Test score: 0.049330148845911026
Test accuracy: 1.0


In [5]:
# 从指定模型保存的位置读取模型，做预测
from tensorflow.keras.models import load_model
import numpy as np

read_model = load_model('./model/irises_classifier_pro.h5')
out = read_model.predict(X_test)
#print("out:%s"%out)

out = tf.nn.softmax(out)
out = np.array(out)

pre_test = np.argmax(out,axis=1)
print('predict:',pre_test)
print('real:',ori_y_test)
print('accuracy:',np.mean(pre_test==ori_y_test))

1/1 [==============================] - 0s 54ms/step
predict: [0 1 0 1 0 0 2 2 2 0 0 2 1 0 2 2 1 0 0 2 2 0 2 1 1 1 2 1 1 2]
real: [0 1 0 1 0 0 2 2 2 0 0 2 1 0 2 2 1 0 0 2 2 0 2 1 1 1 2 1 1 2]
accuracy: 1.0
